# Clustering

In [ ]:
import math
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from scipy.cluster.hierarchy import linkage, dendrogram, cut_tree
from sklearn import tree
from scipy import stats

## Afstanden

In [ ]:
def afstand_euclidisch(tabel, i, j):
    rij1 = tabel.iloc[i]
    rij2 = tabel.iloc[j]
    afstand = math.sqrt(((rij1-rij2)**2).sum())
    return afstand

In [ ]:
tabel = pd.DataFrame({'0': [10,18], '1': [12,14], '2': [15,13], '3': [13,15], '4': [9,17]})
print(tabel)

In [ ]:
afstand_euclidisch(tabel, 0, 1)

In [ ]:
def afstand_manhattan(tabel, i, j):
    rij1 = tabel.iloc[i]
    rij2 = tabel.iloc[j]
    afstand = ((rij1-rij2).abs()).sum()
    return afstand

In [ ]:
afstand_manhattan(tabel, 0, 1)

In [ ]:
tabel = pd.DataFrame({'score1':[648,775,113,938,871,200,227,389,757,561], 'score2':[5,6,6,2,4,7,3,8,1,0]})
print(tabel)

In [ ]:
geschaald = stats.zscore(tabel, ddof=1)
geschaald

In [ ]:
afstand_euclidisch(geschaald, 0, 2)

In [ ]:
afstand_manhattan(geschaald, 0, 2)

## K-means

In [ ]:
voorbeeld = pd.read_csv('voorbeeld.csv')
model = KMeans(n_clusters=4, random_state=42)
model.fit(voorbeeld) # dit voert het hele algoritme uit
clusters = model.labels_ # dit zijn de clusternummers per rij
print(clusters)

In [ ]:
markers = [ 'o', '+', '^', 's'] # marker per cluster
colors = [ 'black', 'green', 'blue', 'red'] # kleur per cluster
fig, ax = plt.subplots()
# plot de clusters
for i in range(0, 4):
    ax.scatter(voorbeeld.x[clusters==i], voorbeeld.y[clusters==i], color=colors[i], marker=markers[i])
# plot de centroids
ax.scatter(model.cluster_centers_[:,0], model.cluster_centers_[:,1], color='purple', marker='D', s=120)
# fig.show()

Je krijgt niet altijd een ideale opdeling:

In [ ]:
voorbeeld = pd.read_csv('voorbeeld.csv')
model = KMeans(n_clusters=4, random_state=1, init='random', n_init=1)
model.fit(voorbeeld) # dit voert het hele algoritme uit
clusters = model.labels_ # dit zijn de clusternummers per rij
fig, ax = plt.subplots()
# plot de clusters
for i in range(0, 4):
    ax.scatter(voorbeeld.x[clusters==i], voorbeeld.y[clusters==i], color=colors[i], marker=markers[i])
# plot de centroids
ax.scatter(model.cluster_centers_[:,0], model.cluster_centers_[:,1], color='purple', marker='D', s=120)
# fig.show()

## Hierarchische clustering

In [ ]:
hclust = pd.read_csv('hclust.csv')
print(hclust)

In [ ]:
fig, ax = plt.subplots()
ax.scatter(hclust.x, hclust.y, color="white")
for i in range(len(hclust)):
    ax.text(x=hclust.x[i], y=hclust.y[i], s=str(i), color='red')
# fig.show()

In [ ]:
fig, ax = plt.subplots()
distances = linkage(hclust, method='single', metric='cityblock')
dgram = dendrogram(distances, labels=hclust.index, ax=ax)
# fig.show()

In [ ]:
clusters = cut_tree(distances, n_clusters=4)
print(clusters[:,0])

In [ ]:
fig, ax = plt.subplots()
ax.scatter(hclust.x, hclust.y, color="white")
for i in range(len(hclust)):
    ax.text(x=hclust.x[i], y=hclust.y[i], s=str(i), color=colors[clusters[i,0]])
# fig.show()

In [ ]:
voorbeeld = pd.read_csv('voorbeeld.csv')
fig, ax = plt.subplots()
distances = linkage(voorbeeld, method='complete', metric='euclidean')
dgram = dendrogram(distances, labels=voorbeeld.index, ax=ax)
# fig.show()

In [ ]:
clusters = cut_tree(distances, n_clusters=4)
print(clusters[:,0])

In [ ]:
clusters = clusters[:,0]
markers = [ 'o', '+', '^', 's'] # marker per cluster
colors = [ 'black', 'green', 'blue', 'red'] # kleur per cluster
fig, ax = plt.subplots()
# plot de clusters
for i in range(4):
    ax.scatter(voorbeeld.x[clusters==i], voorbeeld.y[clusters==i], color=colors[i], marker=markers[i])
# fig.show()

## Combinatie met beslissingsbomen

In [ ]:
target = pd.Series(clusters).apply(str)
data = voorbeeld
model = tree.DecisionTreeClassifier(criterion='entropy', random_state=42)
model.fit(data, target)
fig, ax = plt.subplots()
fig.dpi = 150
tree.plot_tree(model, rounded=True, feature_names=data.columns.tolist(), class_names=model.classes_, fontsize=7.5, ax=ax)
# fig.show()